In [1]:
import json
import datetime
from datetime import timedelta
import os, zipfile
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import chisquare
import numpy as np
import matplotlib.pyplot as plt
from LifeBuddyWebApp import create_app, db
from LifeBuddyWebApp.models import User, Post, Health_description, Health_measure
proxyApp=create_app()
ctx=proxyApp.app_context()
ctx.push()
from sqlalchemy import func

C:\tools\miniconda3\envs\lifebuddy\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [ ]:
#change width of current notebook cells
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
data_zip_med=r"D:\OneDrive\Documents\professional\20210710lifeBuddy\misc_not_for_git\two_traning_sessions.zip"
polar_zip=zipfile.ZipFile(data_zip_med)

#read all files in zip to dictionary of file names and json (each value is a training_session)
polar_data_dict={}
for i in polar_zip.filelist:
    polar_data_dict[i.filename]=json.loads(polar_zip.read(i.filename))

In [ ]:
#get max id from Health_description table
max_id=db.session.query(func.max(Health_description.id)).first()[0]

if max_id==None:
    print('cest none')
    

In [ ]:
df=pd.DataFrame()

In [ ]:
df

In [ ]:
df['new_coloumn2']=[1,2]

In [ ]:
#delete all rows in a table
Health_description.query.delete()
Health_measure.query.delete()
db.session.commit()

In [34]:
max_id=None
test_dict={}
for i,j in polar_data_dict.items():
    if 'training-session-' in i:
        max_id=1 if max_id==None else max_id +1
#         df=pd.DataFrame()
        temp_df_dict={}
        df_list=[]
        for data_item in j['exercises'][0]['samples'].keys():

            if data_item != 'recordedRoute':
                df1=pd.DataFrame()
                df1['id']=[max_id]
                df1['var_activity']=data_item
                
                temp_df_dict[data_item]=df1
                df_list.append(data_item)
            if len(df_list)==1:
                df=df1
            else:
                frames=[j for i,j in temp_df_dict.items()]
                df=pd.concat(frames)
        
        test_dict[i]=df
#                 print('after data_iteM',max_id,i,data_item)
# polar_data_dict[list(polar_data_dict.keys())[2]][0].keys()

In [38]:
list(test_dict.keys())

['training-session-2021-03-25-5833693154-003b4bf2-e18f-46ad-87df-9254d8434062.json',
 'training-session-2021-06-07-6130436929-01765f57-8962-4cb9-8112-2067cd7aada8.json']

In [39]:
test_dict['training-session-2021-06-07-6130436929-01765f57-8962-4cb9-8112-2067cd7aada8.json']

,id,var_activity
0,2,heartRate
0,2,speed
0,2,distance


In [ ]:
pd.DataFrame.from_dict(test_dict, orient='index').to_excel('test_dict.xlsx')

In [1]:
#read all training sessions into df's inside of a dict
polar_df_dict1={}
polar_df_dict2={}
for i,j in polar_data_dict.items():
    if 'training-session-' in i:
        #Make Health_description df
        if max_id==None:
            max_id=1
        else:
            max_id +=1
            for data_item in j['exercises'][0]['samples'].keys():
                # At ths point we have 1,2 or three data sets for each i
                if data_item != 'recordedRoute':

                    # test_dict[i]=max_id
                    
                    df1=pd.DataFrame()
                    df1['id']=[max_id]
                    df1['var_activity']=[j['name']]
                    df1['var_type']=['heart rate']
                    df1['var_periodicity']=['seconds']
                    df1['var_unit']=['heart rate per second']
                    df1['var_timezone_utc_delta_in_mins']=[j['timeZoneOffset']]
                    df1['time_stamp_utc']=[datetime.datetime.utcnow()]
                    df1['user_id']=[1]#replace with current user
                    df1['source_filename']=[i]

                    var_datetime_utc=j['timeZoneOffset']
                    var_datetime_utc_list=[datetime.datetime.strptime(
                        k['dateTime']  ,'%Y-%m-%dT%H:%M:%S.%f') + timedelta(
                            minutes=var_datetime_utc) for k in j['exercises'][0]['samples']['heartRate']]
                    var_values_list=[k['value'] for k in j['exercises'][0]['samples']['heartRate']]
                    df2=pd.DataFrame(list(zip(var_datetime_utc_list,var_values_list)), columns=[
                        'var_datetime_utc','var_value'])
                    df2['description_id']=max_id
                    
                
                polar_df_dict1[i]=df1
                polar_df_dict2[i]=df2

NameError: name 'polar_data_dict' is not defined

In [ ]:
polar_df_dict1[list(polar_df_dict1.keys())[0]]

In [ ]:
polar_data_dict[list(polar_data_dict.keys())[4]]